In [1]:
import numpy as np
import subprocess
import pickle
import time
from ase.spacegroup import crystal
from ase.build import make_supercell
from ase.io.lammpsdata import write_lammps_data, read_lammps_data
import sys
from tqdm import tqdm
from scipy.constants import physical_constants
kB = physical_constants["Boltzmann constant in eV/K"][0]

In [2]:
# Load saved arrays for testing
T=1073
swaps = np.load("test_1/swap_atoms_all_steps.npy")
rands = np.load("test_1/rands_all_steps.npy")
en_stored = np.load("test_1/Eng_all_steps.npy")
accepts = np.load("test_1/accepts_all_steps.npy")
elems = ["Co", "Ni", "Cr", "Fe", "Mn"]

In [3]:
en_stored.shape, accepts.shape

((100,), (100,))

## First make the test lammps command file
### The random seed should be the same as the test, so we'll read it in explicitly

In [4]:
with open("test_1/in_1.minim", "r") as fl:
    commandList = fl.readlines()

# Change the following potential file location line since we're one directory up 
commandList[7] = 'pair_coeff \t * * ../pot/library.meam Co Ni Cr Fe Mn ../pot/params.meam Co Ni Cr Fe Mn\n'

# Let's view the commands for the test command file
for command in commandList:
    print(command)

units 	 metal

atom_style 	 atomic

atom_modify 	 map array

boundary 	 p p p

atom_modify 	 sort 0 0.0

read_data 	 inp_MC_1.data

pair_style 	 meam

pair_coeff 	 * * ../pot/library.meam Co Ni Cr Fe Mn ../pot/params.meam Co Ni Cr Fe Mn

neighbor 	 0.3 bin

neigh_modify 	 delay 0 every 1 check yes

variable x equal pe

displace_atoms all random 0.1 0.1 0.1 5028

minimize		 1e-5 0.0 1000 10000

run 0

print "$x" file Eng_1.txt


In [5]:
with open("in_test.minim", "w") as fl:
    fl.writelines(commandList)

## Check the energies of all the supercells

In [6]:
supercells = []
enComps = np.zeros_like(en_stored)
# start with the initial supercell
with open("test_1/superInitial_1.pkl", "rb") as fl:
    sup0 = pickle.load(fl)
supercells.append(sup0)

# Compute energy
write_lammps_data("inp_MC_1.data", sup0, specorder=elems)
cmd = subprocess.Popen("$LMPPATH/lmp -in in_test.minim > out_test.txt", shell=True)
rt = cmd.wait()
assert rt == 0

# Check energy
with open("Eng_1.txt", "r") as fl:
    en = float(fl.readline().split()[0])
assert np.allclose(en_stored[0], en) 
enComps[0] = en

# Now go through the rest of the supercells
for i in tqdm(range(1, en_stored.shape[0]), position=0, leave=True, ncols=65):
    # Compute energy
    with open("test_1/chkpt/supercell_{}.pkl".format(i), "rb") as fl:
        sup1 = pickle.load(fl)
    
    write_lammps_data("inp_MC_1.data", sup1, specorder=elems)
    cmd = subprocess.Popen("$LMPPATH/lmp -in in_test.minim > out_test.txt", shell=True)
    rt = cmd.wait()
    assert rt == 0
    with open("Eng_1.txt", "r") as fl:
        en = float(fl.readline().split()[0])
    
    # Check energy
    assert np.allclose(en, en_stored[i])
    supercells.append(sup1)
    enComps[i] = en

print("Energy assertions passed")

100%|████████████████████████████| 99/99 [00:58<00:00,  1.69it/s]

Energy assertions passed


## Match the rejected and accepted moves against the random numbers

In [8]:
rejected = []
accepted = []
for enInd in range(en_stored.shape[0] - 1):
    # If the same energy occurs, then move was rejected
    if en_stored[enInd + 1] - en_stored[enInd] == 0.0:
        assert accepts[enInd] == 0
        rejected.append(enInd)
    else:
        assert accepts[enInd] == 1
        accepted.append(enInd)

In [9]:
len(rejected)

26

In [12]:
# Let's first check a swap that was rejected
for move in tqdm(rejected, position=0, leave=True, ncols=65): # index of initial state of the move
    sup_temp = supercells[move].copy()
    tmp = sup_temp[swaps[move, 0]].symbol
    sup_temp[swaps[move, 0]].symbol = sup_temp[swaps[move, 1]].symbol
    sup_temp[swaps[move, 1]].symbol = tmp

    # compute energy
    write_lammps_data("inp_MC_1.data", sup_temp, specorder=elems)
    cmd = subprocess.Popen("$LMPPATH/lmp -in in_test.minim > out_test.txt", shell=True)
    rt = cmd.wait()
    assert rt == 0

    # Read energy
    with open("Eng_1.txt", "r") as fl:
        en_temp = float(fl.readline().split()[0])

    # Check what the random number was
    rand = rands[move]
    de = en_temp - en_stored[move]
    test_num = np.exp(-de/(kB*T))
    # Check that rand is greater than relative prob.
    assert rand > test_num

print("Rejection checks okay.")

100%|████████████████████████████| 26/26 [00:15<00:00,  1.67it/s]

Rejection checks okay.


In [13]:
# Now Let's check the swaps that were accepted
for move in tqdm(accepted, position=0, leave=True, ncols=65):
    sup_temp = supercells[move].copy()
    tmp = sup_temp[swaps[move, 0]].symbol
    sup_temp[swaps[move, 0]].symbol = sup_temp[swaps[move, 1]].symbol
    sup_temp[swaps[move, 1]].symbol = tmp

    # compute energy
    write_lammps_data("inp_MC_1.data", sup_temp, specorder=elems)
    cmd = subprocess.Popen("$LMPPATH/lmp -in in_test.minim > out_test.txt", shell=True)
    rt = cmd.wait()

    # Read energy
    with open("Eng_1.txt", "r") as fl:
        en_temp = float(fl.readline().split()[0])
    
    assert np.allclose(en_temp, en_stored[move + 1])
    assert sup_temp == supercells[move + 1]
    de = en_stored[move + 1] - en_stored[move]

    test_num = np.exp(-de/(kB*T))
    rand = rands[move]

    assert rand < test_num

print("Acceptance tests okay.")

100%|████████████████████████████| 73/73 [00:43<00:00,  1.67it/s]

Acceptance tests okay.
